# Classifier Training

***
## Notes

 - .

## Download exported tables locally for use with TensorFlow

In [1]:
# Authenticate Google Drive access
# following guide: https://gsuitedevs.github.io/PyDrive/docs/build/html/quickstart.html
# Need client_secrets.json file for auth to work -- I put it in 'notebooks' folder, but how can we put it somewhere more appropriate and still have the auth find it?
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth()

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=207798239714-s97urn6kn6eqj2li0l2k8ud9fudlqqbf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [2]:
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)

# list files in root folder
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()

# Get file id of export folder we're saving tables to
exportFolderID = [file['id'] for file in file_list if file['title']=='GEE_exports'][0] 

# lists files in export folder
file_list = drive.ListFile({'q': "'" + exportFolderID + "' in parents and trashed=false"}).GetList()
for file1 in file_list:
    print('title: %s, id: %s' % (file1['title'], file1['id']))
    
file_list[0].GetContentFile('scene1_sample_pts.csv')

title: scene1_sample_pts.csv, id: 1kgY0F0d_s9X852hMUIzMK3Z4W1xavqvw
title: scene1_sample_pts.csv, id: 1OmhVTmifCv4Lo_WC6NvJ7Welf08pwrQW


## Playing with TensorFlow

In [1]:
import numpy as np
import tensorflow as tf

## Random Forest

## Neural Network

## SVM